# Extrair e tratar dados da planilha de respostas do fornecedor #

In [2]:
import pandas as pd
from openpyxl import load_workbook
import json

In [3]:
def extract_info(file_path, sheet_name, column_letters):
    """
    Extracts data from specific columns in an Excel sheet.

    Args:
        file_path (str): Path to the input Excel file.
        sheet_name (str): Name of the sheet to extract data from.
        column_letters (list of str): List of column letters to extract data from.

    Returns:
        pd.DataFrame: DataFrame containing the extracted data with column names from row 15.
    """
    # Load workbook and sheet
    wb = load_workbook(file_path, data_only=True)
    sheet = wb[sheet_name]

    # Read column names from row 15
    column_names = [sheet[f'{col}15'].value for col in column_letters]

    # Initialize a dictionary to store the data
    data = {col: [] for col in column_names}

    # Extract values from each column, ignoring the first 13 cells and row 15
    for col_letter, column_name in zip(column_letters, column_names):
        column_data = [cell.value for cell in sheet[col_letter] if cell.row > 13]
        # Add data to the corresponding list
        data[column_name] = column_data

    # Find the maximum length of the lists
    max_length = max(len(lst) for lst in data.values())

    # Standardize the length of the lists with missing values (None)
    for key in data:
        while len(data[key]) < max_length:
            data[key].append(None)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data)

    return df



Planilhas a serem extraídas:

In [24]:
file_path_khs = '../files/KHS.xlsx'
file_path_krones = '../files/KRONES.xlsx'
file_path_sidel = '../files/SIDEL.xlsx'
sheet_filler = 'CAN Filler'
column_filler = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']

Seções Can Filler:

In [34]:
can_filler_sections = {
    "1.0 PERFORMANCE and WARRANTY": {
        "start": "",
        "end": ""
    },
    "2.0 GENERAL INFORMATION": {
        "start": "",
        "end": ""
    },
    "3.0 PROCESS": {
        "start": "",
        "end": ""
    },
    "4.0 CONSTRUCTIVE CHARACTERISTICS": {
        "start": "",
        "end": ""
    },
    "5.0 FILLER ACCESSORIES": {
        "start": "",
        "end": ""
    },
    "6.0 DIMENSIONS / WEIGHT": {
        "start": "",
        "end": ""
    },
    "7.0 OXYGEN ELIMINATORS DEVICES": {
        "start": "",
        "end": ""
    },
    "8.0 MATERIALS": {
        "start": "",
        "end": ""
    },
    "9.0 CHANGE OVER": {
        "start": "",
        "end": ""
    },
    "10.0 SAFETY": {
        "start": "",
        "end": ""
    },
    "11.0 ELECTRICITY, CONTROL, ALARM, AUTOMATION": {
        "start": "",
        "end": ""
    },
    "12.0 POWER CABINETS": {
        "start": "",
        "end": ""
    },
    "13.0 UTILITIES CONSUMPTION": {
        "start": "",
        "end": ""
    },
    "14.0 MAINTENANCE / DOCUMENTATION / TRAINING": {
        "start": "",
        "end": ""
    },
    "15.0 TEC CAN FILLER ACCEPTANCE": {
        "start": "",
        "end": ""
    },
    "16.0 ZONE DEMANDS DUE TO LOCAL REGULATIONS": {
        "start": "",
        "end": ""
    },
    "17.0 HISTORY OF REVISIONS": {
        "start": "",
        "end": ""
    }
}


Identificar os índices das seções:

In [6]:
def is_section(item):
    return isinstance(item, str) and item in can_filler_sections

In [7]:
def get_section_index(df):
    section_indexes = df[df['ITEM'].apply(is_section)].index.tolist()
    section_indexes.append(len(df))  # adiciona o índice do final do DataFrame
    return section_indexes

In [35]:
def add_indexes(df, indexes, sections_dict):
    for i in range(len(indexes) - 1):
        start_idx = indexes[i]
        end_idx = indexes[i + 1]
        section_name = df.loc[start_idx, 'ITEM']
        sections_dict[section_name]["start"] = start_idx + 1 
        sections_dict[section_name]["end"] = end_idx

Separar planilhas:

In [54]:
def separate_df(df_supplier, sections, section_name):
    df = df_supplier.iloc[sections[section_name]['start']:sections[section_name]['end']].reset_index(drop=True).rename(columns={'CNV': 'Charachteristic', 'UNIT': 'Instruction/Comments'}).drop(index=0, columns=['ITEM', 'ITEM DESCRIPTION', 'INDEX', 'Instruction / Comments', None])
    return df

In [49]:
def separate_df_performance(df_supplier, sections, section_name):
    df = df_supplier.iloc[sections[section_name]['start']:sections[section_name]['end']].reset_index(drop=True).drop(index=[0], columns=['ITEM', None])
    return df

In [43]:
def separate_df_utilities(df_supplier, sections, section_name):
    df = df_supplier.iloc[sections[section_name]['start']:sections[section_name]['end']].reset_index(drop=True).drop(index=[0], columns=['ITEM', 'ITEM DESCRIPTION', 'Instruction / Comments']).rename(columns={'CNV': 'Characteristic', 'INDEX': 'Instruction / Comments'}).dropna(subset=['Characteristic'])
    return df

In [62]:
def separate_df_maintenance(df_supplier, sections, section_name):
    df = df_supplier.iloc[sections[section_name]['start']:sections[section_name]['end']].reset_index(drop=True).drop(index=[0], columns=['ITEM', 'INDEX', 'ITEM DESCRIPTION', 'Instruction / Comments']).rename(columns={'CNV': 'Characteristic', 'UNIT': 'Instruction', None: 'Comments'}).dropna(subset=['Characteristic'])
    return df

In [45]:
def separate_df_zone(df_supplier, sections, section_name):
    df = df_supplier.iloc[sections[section_name]['start']:sections[section_name]['end']].reset_index(drop=True).drop(index=[0], columns=['ITEM', 'UNIT', 'INDEX']).rename(columns={'CNV': 'Zone', 'ITEM DESCRIPTION': 'Detail','Instruction / Comments': 'Instruction', None: 'Comments'})
    return df

# KHS: # 

In [27]:
df_khs = extract_info(file_path=file_path_khs, sheet_name=sheet_filler, column_letters=column_filler)

Índices:

In [ ]:
df_khs.head(50)

In [36]:
khs_indexes = get_section_index(df_khs)
_ = add_indexes(df_khs, khs_indexes, can_filler_sections)
can_filler_sections

{'1.0 PERFORMANCE and WARRANTY': {'start': 1, 'end': 29},
 '2.0 GENERAL INFORMATION': {'start': 30, 'end': 40},
 '3.0 PROCESS': {'start': 41, 'end': 71},
 '4.0 CONSTRUCTIVE CHARACTERISTICS': {'start': 72, 'end': 91},
 '5.0 FILLER ACCESSORIES': {'start': 92, 'end': 139},
 '6.0 DIMENSIONS / WEIGHT': {'start': 140, 'end': 149},
 '7.0 OXYGEN ELIMINATORS DEVICES': {'start': 150, 'end': 154},
 '8.0 MATERIALS': {'start': 155, 'end': 181},
 '9.0 CHANGE OVER': {'start': 182, 'end': 187},
 '10.0 SAFETY': {'start': 188, 'end': 197},
 '11.0 ELECTRICITY, CONTROL, ALARM, AUTOMATION': {'start': 198, 'end': 233},
 '12.0 POWER CABINETS': {'start': 234, 'end': 246},
 '13.0 UTILITIES CONSUMPTION': {'start': 247, 'end': 263},
 '14.0 MAINTENANCE / DOCUMENTATION / TRAINING': {'start': 264, 'end': 290},
 '15.0 TEC CAN FILLER ACCEPTANCE': {'start': 291, 'end': 293},
 '16.0 ZONE DEMANDS DUE TO LOCAL REGULATIONS': {'start': 294, 'end': 319},
 '17.0 HISTORY OF REVISIONS': {'start': 320, 'end': 330}}

Planilhas separadas:

In [70]:
df_performance = separate_df_performance(df_khs, can_filler_sections, "1.0 PERFORMANCE and WARRANTY")
df_general_info = separate_df(df_khs, can_filler_sections, "2.0 GENERAL INFORMATION")
df_process = separate_df(df_khs, can_filler_sections, "3.0 PROCESS")
df_characteristics = separate_df(df_khs, can_filler_sections, "4.0 CONSTRUCTIVE CHARACTERISTICS")
df_accessories = separate_df(df_khs, can_filler_sections, "5.0 FILLER ACCESSORIES")
df_dimensions = separate_df(df_khs, can_filler_sections, "6.0 DIMENSIONS / WEIGHT")
df_oxygen = separate_df(df_khs, can_filler_sections, "7.0 OXYGEN ELIMINATORS DEVICES")
df_materials = separate_df(df_khs, can_filler_sections, "8.0 MATERIALS")
df_change = separate_df(df_khs, can_filler_sections, "9.0 CHANGE OVER")
df_safety = separate_df(df_khs, can_filler_sections, "10.0 SAFETY")
df_electricity = separate_df(df_khs, can_filler_sections, "11.0 ELECTRICITY, CONTROL, ALARM, AUTOMATION")
df_power = separate_df(df_khs, can_filler_sections, "12.0 POWER CABINETS")
df_utilities = separate_df_utilities(df_khs, can_filler_sections, "13.0 UTILITIES CONSUMPTION")
df_maintenance = separate_df_maintenance(df_khs, can_filler_sections, "14.0 MAINTENANCE / DOCUMENTATION / TRAINING")
df_acceptance = separate_df(df_khs, can_filler_sections, "15.0 TEC CAN FILLER ACCEPTANCE")
df_zone = separate_df_zone(df_khs, can_filler_sections, "16.0 ZONE DEMANDS DUE TO LOCAL REGULATIONS")

dataframes = [
    df_performance,
    df_general_info,
    df_process,
    df_characteristics,
    df_accessories,
    df_dimensions,
    df_oxygen,
    df_materials,
    df_change,
    df_safety,
    df_electricity,
    df_power,
    df_utilities,
    df_maintenance,
    df_acceptance,
    df_zone
]


# Chamar o LLM #

In [67]:
import os
from dotenv import load_dotenv

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
from langchain_openai import AzureChatOpenAI

load_dotenv()

True

In [68]:
model = AzureChatOpenAI(
            azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
            openai_api_key = os.environ["AZURE_OPENAI_API_KEY"],
            deployment_name = os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
            api_version = "2023-09-01-preview",
            temperature = 0.0
        )

In [69]:
prompt_template = """
    You are an expert at selecting suppliers that will provide equipments to the company you work for.
    The suppliers fill a spreadsheet with their machines' specifications and based on that, you check if the answers correspond to what your company requires to make them an official supplier.

    In this task, you will analyze this data:
    
    ```
    {df_data}
    ```

    For each JSON object, you will check if the supplier's answer can fill the requirements according to the other fields.
    The CNV field describes what is being analyzed, in case you need more information.

    GUIDELINES:
    - For each JSON object, you should add another field called "COMMENT" and it should only contain "OK" or "NOK". "OK" in case the supplier's answer can fill the requirement or "NOK" in case the supplier's answer does not fill the requirement.
    - For each JSON object, you should add another field called "REASON", and it should contain the reason for you to label the answer as "OK" or "NOK".
    - Your response should only contain a valid JSON with the analysis made.
    - If there's not enough information to make the analysis, in the field "COMMENT" just write "Not enough information".
    - Don't evaluate the suppliers answers if you don't know if they fill the requirements.
    """